In [1]:
import pandas as pd

data = {
    'ticket': [
        "My internet is down since morning.",
        "I want to change my billing address.",
        "How do I reset my account password?",
        "My service was disconnected unexpectedly.",
        "Need help upgrading my plan."
    ]
}

df = pd.DataFrame(data)


In [2]:
TAGS = ["Billing", "Technical", "Account", "Service", "Upgrade"]


In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="ghp_xxxxx",  # because of security risk its removed replace it with your GitHub token
    base_url="https://models.github.ai/inference/v1"
)

def classify_zero_shot(text, tags):
    prompt = f"""
You are a support ticket classification assistant.
Given the ticket below, choose the top 3 most relevant categories from: {", ".join(tags)}.

Ticket: "{text}"

Return the top 3 categories as a comma-separated list.
"""
    response = client.chat.completions.create(
        model="openai/gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()


In [11]:
df['predicted_tags_zero_shot'] = df['ticket'].apply(lambda x: classify_zero_shot(x, TAGS))
print(df[['ticket', 'predicted_tags_zero_shot']])


                                      ticket     predicted_tags_zero_shot
0         My internet is down since morning.  Technical, Service, Account
1       I want to change my billing address.    Billing, Account, Service
2        How do I reset my account password?  Account, Technical, Service
3  My service was disconnected unexpectedly.  Service, Technical, Account
4               Need help upgrading my plan.    Upgrade, Account, Service


In [12]:
TAGS = ["Billing", "Technical", "Account", "Service", "Upgrade"]

sample_ticket = "My internet was cut off without any notice. Please fix this."

result = classify_zero_shot(sample_ticket, TAGS)
print(result)  # e.g., "Service, Technical, Account"


Service, Technical, Account


In [15]:
FEW_SHOT_PROMPT = """
You are a ticket classification assistant. Choose the top 3 tags from:
Billing, Technical, Account, Service, Upgrade

Examples:
Ticket: "I was overcharged on my last bill."
Tags: Billing, Account, Service

Ticket: "Internet not working for 2 days."
Tags: Technical, Service, Account

Now classify:
Ticket: "{text}"
Tags:
"""

def classify_few_shot(text):
    prompt = FEW_SHOT_PROMPT.format(text=text)
    response = client.chat.completions.create(
        model="openai/gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()


In [16]:
df['predicted_tags_few_shot'] = df['ticket'].apply(classify_few_shot)
print(df[['ticket', 'predicted_tags_zero_shot', 'predicted_tags_few_shot']])


                                      ticket     predicted_tags_zero_shot  \
0         My internet is down since morning.  Technical, Service, Account   
1       I want to change my billing address.    Billing, Account, Service   
2        How do I reset my account password?  Account, Technical, Service   
3  My service was disconnected unexpectedly.  Service, Technical, Account   
4               Need help upgrading my plan.    Upgrade, Account, Service   

       predicted_tags_few_shot  
0  Technical, Service, Account  
1    Billing, Account, Service  
2  Account, Technical, Service  
3  Service, Account, Technical  
4    Upgrade, Account, Service  


In [17]:
print(df)

                                      ticket     predicted_tags_zero_shot  \
0         My internet is down since morning.  Technical, Service, Account   
1       I want to change my billing address.    Billing, Account, Service   
2        How do I reset my account password?  Account, Technical, Service   
3  My service was disconnected unexpectedly.  Service, Technical, Account   
4               Need help upgrading my plan.    Upgrade, Account, Service   

       predicted_tags_few_shot  
0  Technical, Service, Account  
1    Billing, Account, Service  
2  Account, Technical, Service  
3  Service, Account, Technical  
4    Upgrade, Account, Service  
